In [1]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import pandas as pd
%matplotlib inline



In [2]:
def load_chemkin_file(path): 
    """ 
    Load Chemkin file and extract the reactions and species.
    Path should be the path to the species folder.
    """
    
    full_path = os.path.join(path,'chemkin')
    chemkin_path = os.path.join(full_path,'copies', 'copy_chem_annotated.inp')
   # chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')

    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path, check_duplicates=False)

    return species_list, reaction_list

In [12]:
############# load the chemkin file and the cantera dataframe ################

#file = 'sensitivites_copy_chem_annotated.cti.csv'
#species_list, reaction_list = load_chemkin_file('/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/methane_with_added_2_BTP/cantera/Nora/2_BTP'
#this is the same as below 

chemkin_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_NK/copy_chem_annotated.inp'
dictionary_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_NK/species_dictionary.txt'
transport_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_NK/tran.dat'


species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path,check_duplicates=False)
#df = pd.read_csv('/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/NIST_with_Ms_sensitivity/2-BTP_kinetics_with_M.cti_sensitivities.csv')

In [6]:
#we want to focus on the top ___ most sensitive reactions 
n = 20

#use the csv files in this directory
list_of_csv_files = [file for file in os.listdir('/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/NIST') if re.search('\.csv',file)]
#also keep in mind that we already flipped the reactions of the following RMG reaction indices
already_flipped = [1, 746, 33, 30, 357, 374, 685, 58, 38, 91, 13, 37, 34, 412, 15, 97, 16, 763, 94, 4, 124, 123, 104, 163, 117, 411,1030,245,558,663]

## flipping kinetics

Using the sensitivity for NIST with Ms, lets find the most sensitive reactions in the 2-BTP NIST model and then change the kinetics of these reactions in RMG's 2_BTP model. 

In [10]:
#let's find a match for the Cantera reactions in sensitvity analysis and then flip them the RMG model to have NIST kinetics
def find_matches(csv_file):
    
    df = pd.read_csv(f'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/NIST/{csv_file}')
    
    RMG_rxn_match = []
    RMG_rxn_match_indices = []
    
    for index,row in df.head(n).iterrows(): #Only iterate over the first n rows


        i=0 # counter, for assertion error later 

        ct_index, sensitivity, ct_equation, ct_products, ct_reactants = row['cantera_index'], row['sensitivity'], row['cantera equation'], eval(row['cantera products']), eval(row['cantera reactants'])

        #list of all products in each reaction 
        list_of_products = [(k+',')*int(v) for k,v in ct_products.items()]   #use ',' as a marker for distinction between different species
        products = set(list(filter(None, (''.join(list_of_products)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

        #list of all reactants in each reaction 
        list_of_reactants = [(k+',')*int(v) for k,v in ct_reactants.items()]   #use ',' as a marker for distinction between different species
        reactants= set(list(filter(None, (''.join(list_of_reactants)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

        #print(products, reactants)
        #iterate through rmg reactions and find the match to the cantera reaction
        for index, rmg_rxn in enumerate(reaction_list): 


                try: 
                    rmg_reactants = set([i for i in rmg_rxn.to_cantera().reactants.keys()])
                    rmg_products = set([i for i in rmg_rxn.to_cantera().products.keys()])

                    #to correct a difference in labeling between RMG and NIST. CH2(T) and CH2* are the same 
                    if ('CH2*' in reactants) or ('CH2*' in products):  
                        if 'CH2(T)' in rmg_reactants:
                            rmg_reactants.remove('CH2(T)') 
                            rmg_reactants.add('CH2*')
                        if 'CH2(T)'in rmg_products:
                            rmg_products.remove('CH2(T)') 
                            rmg_products.add('CH2*')

                    if (reactants==rmg_reactants) and (products==rmg_products):
                        rmg_rxn_to_be_changed=rmg_rxn
                        print(rmg_rxn)
                        print(ct_equation)
                        print(rmg_rxn.index)
                        RMG_rxn_match.append(rmg_rxn)
                        RMG_rxn_match_indices.append(rmg_rxn.index)
                        print('***************************************************************')
                        i += 1 #to check if match was found 
                        break

                    #if equation is written backwards 
                    if (reactants==rmg_products) and (products==rmg_reactants):
                        rmg_rxn_to_be_changed=rmg_rxn
                        print(rmg_rxn)
                        print(ct_equation)
                        print(f'backwards, {rmg_rxn.index}')
                        RMG_rxn_match.append(rmg_rxn)
                        RMG_rxn_match_indices.append(rmg_rxn.index)
                        print('**************************************')
                        i += 1 #to check if match was found 
                        break

                except AttributeError: #some of the rmg_rxn.to_cantera() is a list of rxns
                    for rxn in rmg_rxn.to_cantera(): 
                        rmg_reactants = set([i for i in rxn.to_cantera().reactants.keys()])
                        rmg_products = set([i for i in rxn.to_cantera().products.keys()])

                        if (reactants==rmg_reactants) and (products==rmg_products):
                            rmg_rxn_to_be_changed=rmg_rxn
                            print(rmg_rxn.index)
                            RMG_rxn_match.append(rmg_rxn)
                            RMG_rxn_match_indices.append(rmg_rxn.index)
                            i += 1 #to check if match was found 
                            break


        #if you can't find a match, tell me that 
        if i!=1:
            print(f"Could not find match for NIST reaction: {ct_equation}")
            print('**************************************')
    return RMG_rxn_match, RMG_rxn_match_indices 

        #if you can find a match, make a new chemkin file with the RMG kinetics replaced by the NIST kinetics. 
        #NOTE: NOT PLAYING WITH THERMO

        #feel like this script will take too long, so will just change chemkins by hand. 






In [13]:
##### execute 

RMG_indices_not_matched_yet = []
RMG_rxns_not_matched_yet = []

for file in list_of_csv_files: 
    RMG_rxn_match, RMG_rxn_match_indices = find_matches(file)
    for rxn_index in RMG_rxn_match_indices: 
        if (rxn_index not in already_flipped) and (rxn_index not in RMG_indices_not_matched_yet): 
            RMG_indices_not_matched_yet.append(rxn_index)
            for rxn in RMG_rxn_match:
                if rxn.index==rxn_index:
                    RMG_rxns_not_matched_yet.append(rxn)

O2(4) + H(8) <=> O(9) + OH(2)
H + O2 <=> O + OH
1
***************************************************************
OH(2) + CO(15) <=> H(8) + CO2(16)
CO + OH <=> CO2 + H
30
***************************************************************
HCO(17) <=> H(8) + CO(15)
HCO + M <=> CO + H + M
33
***************************************************************
O2(4) + H(8) <=> HO2(13)
H + O2 (+M) <=> HO2 (+M)
357
***************************************************************
O2(4) + HCO(17) <=> HO2(13) + CO(15)
HCO + O2 <=> CO + HO2
38
***************************************************************
HO2(13) + CH3(19) <=> OH(2) + CH3O(27)
CH3 + HO2 <=> CH3O + OH
97
***************************************************************
H(8) + HCO(17) <=> H2(10) + CO(15)
H + HCO <=> CO + H2
34
***************************************************************
Could not find match for NIST reaction: H2O + HCO <=> CO + H + H2O
**************************************
O2(4) + CH2(T)(18) <=> H(8) + OH(2) + CO(15)
CH

In [15]:
RMG_indices_not_matched_yet

[377, 93, 2, 19, 12, 125]